# 基于AutoEncoder训练wifi指纹的室内定位系统

## 1.获取数据

```shell
    wget https://archive.ics.uci.edu/ml/machine-learning-databases/00310/UJIndoorLoc.zip
    unzip UJIndoorLoc.zip
```

## 2.数据预处理

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

train_data = pd.read_csv('UJIndoorLoc/trainingData.csv', header=0)
test_data  = pd.read_csv('UJIndoorLoc/validationData.csv', header=0)

train_data.head()
# test_data.head()
train_x = scale(np.asarray(train_data.ix[:, 0:520]))
train_y = np.asarray(train_data['BUILDINGID'].map(str) + train_data['FLOOR'].map(str))
train_y = np.asarray(pd.get_dummies(train_y))

test_x = scale(np.asarray(test_data.ix[:, 0:520]))
test_y = np.asarray(test_data['BUILDINGID'].map(str) + test_data['FLOOR'].map(str))
test_y = np.asarray(pd.get_dummies(test_y))

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


## 3.使用AutoEncoder网络训练数据

### 3.1.引入依赖包

In [15]:
import numpy as np
import tensorflow as tf
sess = tf.InteractiveSession()

### 3.2.定义网络参数

In [8]:
input_size  = train_x.shape[1]
num_classes = train_y.shape[1]
X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, num_classes])

Encoders = [(input_size, 256), (256, 128), (128, 64)]
Decoders = [(64, 128), (128, 256), (256, 520)]
dnns     = [(64, 128), (128, 128), (128, num_classes)]

epoch = 30
batch_size = 10
batch_nums = train_x.shape[0] / batch_size

In [26]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=[shape]))

def AutoEncoder(x):
    # Encoder
    encoder_output = x
    for idx, shape in enumerate(Encoders):
        with tf.name_scope('encoder_%d' % idx):
            w = weight_variable(shape)
            b = bias_variable(shape[-1])
            encoder_output = tf.tanh(tf.add(tf.matmul(encoder_output, w), b))
    
    # Decoder 
    o_decoder = encoder_output
    for idx, shape in enumerate(Decoders):
        with tf.name_scope('decoder_%d' % idx):
            w = weight_variable(shape)
            b = bias_variable(shape[-1])
            o_decoder = tf.tanh(tf.add(tf.matmul(o_decoder, w), b))
    decoder_output = o_decoder
    
    # DNN
    o_dnn = encoder_output
    for idx, shape in enumerate(dnns):
        with tf.name_scope('dnn_%d' % idx):
            w = weight_variable(shape)
            b = bias_variable(shape[-1])
            o_dnn = tf.tanh(tf.add(tf.matmul(o_dnn, w), b))
    dnn_output = tf.nn.softmax(o_dnn)
    
    return decoder_output, dnn_output

### 3.3.训练神经网络

In [27]:
def train(x, y):
    decoder_output, dnn_output = AutoEncoder(x)
    
    decoder_cost = tf.reduce_mean(tf.pow(x - decoder_output, 2))
    decoder_opti = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(decoder_cost)
    
    dnn_cost     = - tf.reduce_sum(y*tf.log(dnn_output))
    dnn_opti     = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(dnn_cost)
    
    correct_pred = tf.equal(tf.argmax(dnn_output, 1), tf.argmax(y, 1))
    accuracy     = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    sess.run(tf.global_variables_initializer())

    # 1.we first train the autoencoder
    for e in range(epoch):
        epoch_loss = []
        for batch in range(batch_nums):
            start, end = batch*batch_size, (batch+1)*batch_size
            batch_x, batch_y = train_x[start:end, :], train_y[start:end, :]
            c, _ = sess.run([decoder_cost, decoder_opti], feed_dict={X: batch_x, Y: batch_y})
            epoch_loss.append(c)
        print 'In AutoEncoder, Epoch {}/{}--Loss:{}'.format(e+1, epoch, np.mean(epoch_loss))
    
    # 2.we train dnn next
    for e in range(epoch):
        epoch_loss = []
        for batch in range(batch_nums):
            start, end = batch*batch_size, (batch+1)*batch_size
            batch_x, batch_y = train_x[start:end, :], train_y[start:end, :]
            c, _ = sess.run([dnn_cost, dnn_opti], feed_dict={X: batch_x, Y: batch_y})
            epoch_loss.append(c)
        train_accuracy = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
        test_accuracy = sess.run(accuracy, feed_dict={X: test_x, Y: test_y})
        print 'In DNN, Epoch {}/{}--Loss:{}--Train Accuracy:{}--Test Accuracy:{}'.format(e+1, epoch, np.mean(epoch_loss), train_accuracy, test_accuracy)
        
train(X, Y)

In AutoEncoder, Epoch 1/30--Loss:0.954828560352
In AutoEncoder, Epoch 2/30--Loss:0.871471881866
In AutoEncoder, Epoch 3/30--Loss:0.831469833851
In AutoEncoder, Epoch 4/30--Loss:0.809065341949
In AutoEncoder, Epoch 5/30--Loss:0.794841885567
In AutoEncoder, Epoch 6/30--Loss:0.784694254398
In AutoEncoder, Epoch 7/30--Loss:0.776862323284
In AutoEncoder, Epoch 8/30--Loss:0.770530581474
In AutoEncoder, Epoch 9/30--Loss:0.76526671648
In AutoEncoder, Epoch 10/30--Loss:0.760809540749
In AutoEncoder, Epoch 11/30--Loss:0.756984472275
In AutoEncoder, Epoch 12/30--Loss:0.753665030003
In AutoEncoder, Epoch 13/30--Loss:0.750754833221
In AutoEncoder, Epoch 14/30--Loss:0.748178303242
In AutoEncoder, Epoch 15/30--Loss:0.745875239372
In AutoEncoder, Epoch 16/30--Loss:0.74379837513
In AutoEncoder, Epoch 17/30--Loss:0.74190980196
In AutoEncoder, Epoch 18/30--Loss:0.740180075169
In AutoEncoder, Epoch 19/30--Loss:0.738585531712
In AutoEncoder, Epoch 20/30--Loss:0.737107396126
In AutoEncoder, Epoch 21/30--Los